## Install fluxgapfill

only needed for the first time

## Import fluxgapfill

In [1]:
import pandas as pd
from fluxgapfill.main import *

In [2]:
data_dir = '/home/otto/data/atmos-flux-data/methane-gapfill/data_dir/'

project_date = '20240202'
predictor_set_names = ['baseline', 'basesoil', 'basemet', 'baseflux', 'basefluxvar', 'basemetsoil', 'basefluxsoil', 'all']
sites = [f'ATMOS_{project_date}_{predictor_set_name}' for predictor_set_name in predictor_set_names]

models=['rf','ann']

predictors = ['DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure', # baseline
              'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,Tsoil_mean,SWC_mean', # base + soil
              'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir', # base + met
              'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,NEE_f,Reco,GPP_f,LE_f,H_f', # base + fluxes
              'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,NEE_f,Reco,GPP_f,LE_f,H_f,ch4_mean,ch4_var,co2_var,h2o_var', # base + fluxes + variances + ch4_mean
              'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir,Tsoil_mean,SWC_mean', # base + met + soil
              'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,NEE_f,Reco,GPP_f,LE_f,H_f,Tsoil_mean,SWC_mean', # base + flux + soil
              'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir,Tsoil_mean,SWC_mean,NEE_f,Reco,GPP_f,LE_f,H_f,ch4_mean,ch4_var,co2_var,h2o_var' # all: base + met + soil + fluxes
             ]

for i in range(1,len(sites)):
    print(sites[i])
    print(predictors[i])

ATMOS_20240202_basesoil
DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,Tsoil_mean,SWC_mean
ATMOS_20240202_basemet
DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir
ATMOS_20240202_baseflux
DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,NEE_f,Reco,GPP_f,LE_f,H_f
ATMOS_20240202_basefluxvar
DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,NEE_f,Reco,GPP_f,LE_f,H_f,ch4_mean,ch4_var,co2_var,h2o_var
ATMOS_20240202_basemetsoil
DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir,Tsoil_mean,SWC_mean
ATMOS_20240202_basefluxsoil
DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,NEE_f,Reco,GPP_f,LE_f,H_f,Tsoil_mean,SWC_mean
ATMOS_20240202_all
DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir,Tsoil_mean,SWC_mean,NEE_f,Reco,GPP_f,LE_f,H_f,ch4_mean,ch4_var,co2_var,h2o_var


In [ ]:
for i in range(1,len(sites)):
# for i in range(len(sites)):
    print(f'Site: {sites[i]}')
    print(f'Predictors: {predictors[i]}')

    preprocess(sites=sites[i], 
               na_values=-9999, # set a different na_values if needed
               data_dir=data_dir)

    train(sites=sites[i],
          data_dir=data_dir,
          models=models,
          predictors=predictors[i],
          overwrite_existing_models=True)

    test(data_dir=data_dir,
         sites=sites[i],
         models=models,
         predictors='all',
         split='test',
         distribution='laplace',
         overwrite_results=True
         )

    fp=f'{data_dir}{sites[i]}/gap.csv'
    gap_df = pd.read_csv(fp)
    gap_df = gap_df.fillna(-9999)
    gap_df.to_csv(fp, index = False)

    gapfill(data_dir=data_dir, 
            sites=sites[i],
            models=models)

Site: ATMOS_20240202_basesoil
Predictors: DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,Tsoil_mean,SWC_mean
Data preprocessing... - site: ATMOS_20240202_basesoil
 - Loading site data


/home/otto/git/methane-gapfill-ml/fluxgapfill/preprocess/load.py:9: DtypeWarning: Columns (251) have mixed types. Specify dtype option on import or set low_memory=False.
  site_data = pd.read_csv(site_data_path, na_values=na_values)
  0%|                                                   | 0/100 [00:00<?, ?it/s]

 - Estimating artificial gap distribution...


100%|███████████████████████████████████████| 100/100 [1:04:19<00:00, 38.60s/it]


 - Done estimating artificial gap distribution.
 - Done preprocessing data for site ATMOS_20240202_basesoil.
 - Processed data written to /home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20240202_basesoil.

Model training...
 - site: ATMOS_20240202_basesoil
 - model: rf
 - predictors: DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,Tsoil_mean,SWC_mean
 - Training on 0/10...
 - Training on 1/10...


In [3]:
from fluxgapfill import *

sites = 'ATMOS_20230422_baseline'

'''
1) for testing, input "TEST"
2) for one site, input the name of the site data folder, e.g., "MPM"
3) for multiple sites, input a series of sites devided by ",", e.g., "MPM, rf"
''' 

models=['rf','ann']

'''
1) input a list of model names devided by ",", e.g., ['lasso', 'ann']
models supported: 'lasso', 'ann', 'rf'

'''
# fp=f'{data_dir}{sites}/raw.csv'
# df = pd.read_csv(fp, nrows=1)
# df[df.columns[df.columns.str.contains('u.')]]

# predictors = 'DOY,sin_DOY,cos_DOY,H,LE,co2_flux,h2o_flux,air_temperature,air_pressure,RH,VPD,wind_speed,wind_dir,Tsoil_mean,SWC_mean,SW_IN_POT' # 2023-04-05
predictors = 'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure' # baseline
# predictors = 'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,Tsoil_mean,SWC_mean' # base + soil
# predictors = 'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,NEE_f,Reco,GPP_f,LE,H' # base + fluxes
# predictors = 'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir' # base + met
# predictors = 'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir,Tsoil_mean,SWC_mean' # base + met + soil
# predictors = 'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,NEE_f,Reco,GPP_f,LE,H,Tsoil_mean,SWC_mean' # base + flux + soil
# predictors = 'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir,Tsoil_mean,SWC_mean,NEE_f,Reco,GPP_f,LE,H' # all: base + met + soil + fluxes
'''
1) input a list of predictors. Comma-separated list of predictors. Ignored if
                                predictors_path is provided.
                                Certain keyword predictors are used to denote
                                specific sets of predictors:

2) input shortcuts. 
"all": all usable predictors in the data
"temporal": 

3) input a list of paths to files having predictor names. Comma-separated list path file(s) with
                                      predictor names. See predictors/metereological.txt for an
                                      example.
'''

'\n1) input a list of predictors. Comma-separated list of predictors. Ignored if\n                                predictors_path is provided.\n                                Certain keyword predictors are used to denote\n                                specific sets of predictors:\n\n2) input shortcuts. \n"all": all usable predictors in the data\n"temporal": \n\n3) input a list of paths to files having predictor names. Comma-separated list path file(s) with\n                                      predictor names. See predictors/metereological.txt for an\n                                      example.\n'

## Step-by-step Gapfilling

In [6]:
project_date = '20240208'
predictor_set_names = ['baseline', 'basesoil', 'basemet', 'baseflux', 'basefluxvar', 'basemetsoil', 'basefluxsoil', 'all']
sites = [f'ATMOS_{project_date}_{predictor_set_name}' for predictor_set_name in predictor_set_names]

models=['rf','ann']

predictors = ['DOY,sin_DOY,cos_DOY,TA,SW_IN_POT,WS,PA', # baseline
              'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,Tsoil_mean,SWC_mean', # base + soil
              'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir', # base + met
              'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,NEE_f,Reco,GPP_f,LE_f,H_f', # base + fluxes
              'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,NEE_f,Reco,GPP_f,LE_f,H_f,ch4_mean,ch4_var,co2_var,h2o_var', # base + fluxes + variances + ch4_mean
              'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir,Tsoil_mean,SWC_mean', # base + met + soil
              'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,NEE_f,Reco,GPP_f,LE_f,H_f,Tsoil_mean,SWC_mean', # base + flux + soil
              'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir,Tsoil_mean,SWC_mean,NEE_f,Reco,GPP_f,LE_f,H_f,ch4_mean,ch4_var,co2_var,h2o_var' # all: base + met + soil + fluxes
             ]


In [6]:
preprocess(sites=sites, 
           na_values=-9999, # set a different na_values if needed
           data_dir=data_dir)

Data preprocessing... - site: ATMOS_20230422_baseline
 - Loading site data


/home/otto/git/methane-gapfill-ml/fluxgapfill/preprocess/load.py:9: DtypeWarning: Columns (251) have mixed types. Specify dtype option on import or set low_memory=False.
  site_data = pd.read_csv(site_data_path, na_values=na_values)
  0%|                                                                                                                                                   | 0/100 [00:00<?, ?it/s]

 - Estimating artificial gap distribution...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [29:09<00:00, 17.50s/it]


 - Done estimating artificial gap distribution.
 - Done preprocessing data for site ATMOS_20230422_baseline.
 - Processed data written to /home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230422_baseline.



In [ ]:
train(sites=sites[i],
      data_dir=data_dir,
      models=models,
      predictors=predictors[i],
      overwrite_existing_models=True)

test(data_dir=data_dir,
     sites=sites[i],
     models=models,
     predictors='all',
     split='test',
     distribution='laplace',
     overwrite_results=True
     )

fp=f'{data_dir}{sites[i]}/gap.csv'
gap_df = pd.read_csv(fp)
gap_df = gap_df.fillna(-9999)
gap_df.to_csv(fp, index = False)

gapfill(data_dir=data_dir, 
        sites=sites[i],
        models=models)

Model training...
 - site: ATMOS_20240208_baseline
 - model: rf
 - predictors: DOY,sin_DOY,cos_DOY,TA,SW_IN_POT,WS,PA
 - Training on 0/10...
 - Training on 1/10...
 - Training on 2/10...
 - Training on 3/10...
 - Training on 4/10...
 - Training on 5/10...
 - Training on 6/10...
 - Training on 7/10...
 - Training on 8/10...
 - Training on 9/10...
 - Done model training.

Model training...
 - site: ATMOS_20240208_baseline
 - model: ann
 - predictors: DOY,sin_DOY,cos_DOY,TA,SW_IN_POT,WS,PA
 - Training on 0/10...


/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.

 - Training on 1/10...


/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.

 - Training on 2/10...


...

/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


 - Training on 7/10...


/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.

 - Training on 8/10...


/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.

In [20]:
for i in range(len(sites)):
    print(f'Site: {sites[i]}')
    print(f'Predictors: {predictors[i]}')

    preprocess(sites=sites[i], 
               na_values=-9999, # set a different na_values if needed
               data_dir=data_dir)

    train(sites=sites[i],
          data_dir=data_dir,
          models=models,
          predictors=predictors[i],
          overwrite_existing_models=True)

    test(data_dir=data_dir,
         sites=sites[i],
         models=models,
         predictors='all',
         split='test',
         distribution='laplace',
         overwrite_results=True
         )

    fp=f'{data_dir}{sites[i]}/gap.csv'
    gap_df = pd.read_csv(fp)
    gap_df = gap_df.fillna(-9999)
    gap_df.to_csv(fp, index = False)

    gapfill(data_dir=data_dir, 
            sites=sites[i],
            models=models)

## Create or Specify the directory of a Project Folder

The project folder should/will contain all input data and computational results.

In [5]:
data_dir = '/home/otto/data/atmos-flux-data/methane-gapfill/data_dir/'

## Prepare Site Data

For testing - no need to prepare data. 

For gapfilling site data,

1) create a site data folder under the Project folder

2) set the folder name to "site name" 

3) organize the site data in a .csv file and name it as "raw.csv"

4) put the site data csv in Project Folder/Site Data Folder


## Set Parameters

In [6]:
preprocess(sites=sites, 
           na_values=-9999, # set a different na_values if needed
           data_dir=data_dir)

Data preprocessing... - site: ATMOS_20230422_baseline
 - Loading site data


/home/otto/git/methane-gapfill-ml/fluxgapfill/preprocess/load.py:9: DtypeWarning: Columns (251) have mixed types. Specify dtype option on import or set low_memory=False.
  site_data = pd.read_csv(site_data_path, na_values=na_values)
  0%|                                                                                                                                                   | 0/100 [00:00<?, ?it/s]

 - Estimating artificial gap distribution...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [29:09<00:00, 17.50s/it]


 - Done estimating artificial gap distribution.
 - Done preprocessing data for site ATMOS_20230422_baseline.
 - Processed data written to /home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230422_baseline.



In [7]:
train(sites=sites,
      data_dir=data_dir,
      models=models,
      predictors=predictors,
      overwrite_existing_models=True)

test(data_dir=data_dir,
     sites=sites,
     models=models,
     predictors='all',
     split='test',
     distribution='laplace',
     overwrite_results=True
     )

fp=f'{data_dir}{sites}/gap.csv'
gap_df = pd.read_csv(fp)
gap_df = gap_df.fillna(-9999)
gap_df.to_csv(fp, index = False)

gapfill(data_dir=data_dir, 
        sites=sites,
        models=models)

/home/otto/git/methane-gapfill-ml/fluxgapfill/train/train.py:85: DtypeWarning: Columns (251) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(train_dir / f"train{i+1}.csv")
/home/otto/git/methane-gapfill-ml/fluxgapfill/train/train.py:85: DtypeWarning: Columns (251) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(train_dir / f"train{i+1}.csv")
/home/otto/git/methane-gapfill-ml/fluxgapfill/train/train.py:85: DtypeWarning: Columns (251) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(train_dir / f"train{i+1}.csv")
/home/otto/git/methane-gapfill-ml/fluxgapfill/train/train.py:85: DtypeWarning: Columns (251) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(train_dir / f"train{i+1}.csv")
/home/otto/git/methane-gapfill-ml/fluxgapfill/train/train.py:85: DtypeWarning: Columns (251) have mixed types. Specify dtype option on import or set low

Model training...
 - site: ATMOS_20230422_baseline
 - model: rf
 - predictors: DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure
 - Training on 0/10...
 - Training on 1/10...
 - Training on 2/10...
 - Training on 3/10...
 - Training on 4/10...
 - Training on 5/10...
 - Training on 6/10...
 - Training on 7/10...
 - Training on 8/10...
 - Training on 9/10...
 - Done model training.

Model training...
 - site: ATMOS_20230422_baseline
 - model: ann
 - predictors: DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure
 - Training on 0/10...
 - Training on 1/10...
 - Training on 2/10...
 - Training on 3/10...
 - Training on 4/10...
 - Training on 5/10...
 - Training on 6/10...
 - Training on 7/10...


/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


 - Training on 8/10...
 - Training on 9/10...
 - Done model training.

Model testing...
 - site: ATMOS_20230422_baseline
 - model: rf
 - predictors: predictors
 - Writing test metrics to: /home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230422_baseline/models/rf/predictors/test_results.csv

Model testing...
 - site: ATMOS_20230422_baseline
 - model: ann
 - predictors: predictors
 - Writing test metrics to: /home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230422_baseline/models/ann/predictors/test_results.csv

Gapfilling: ATMOS_20230422_baseline, rf, predictors, laplace


/home/otto/git/methane-gapfill-ml/fluxgapfill/gapfill/gapfill.py:176: DtypeWarning: Columns (251) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv(raw_fn)


 - Writing gapfilled data to /home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230422_baseline/gapfilled/rf_predictors_laplace.csv

Computing Annual Budget: ATMOS_20230422_baseline, rf, predictors, laplace
 - Budget between 2022-01-01 and 2023-01-01 is -0.00 +/- 0.00
 - Budget between 2023-01-01 and 2024-01-01 is -0.00 +/- 0.00
 - Writing budget data to /home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230422_baseline/gapfilled/rf_predictors_laplace_budget.csv

Gapfilling: ATMOS_20230422_baseline, ann, predictors, laplace


/home/otto/git/methane-gapfill-ml/fluxgapfill/gapfill/gapfill.py:176: DtypeWarning: Columns (251) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv(raw_fn)


 - Writing gapfilled data to /home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230422_baseline/gapfilled/ann_predictors_laplace.csv

Computing Annual Budget: ATMOS_20230422_baseline, ann, predictors, laplace
 - Budget between 2022-01-01 and 2023-01-01 is -0.01 +/- 4.44
 - Budget between 2023-01-01 and 2024-01-01 is -0.00 +/- 0.83
 - Writing budget data to /home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230422_baseline/gapfilled/ann_predictors_laplace_budget.csv



## One-click Gapfilling

In [17]:
run_all(data_dir=data_dir, 
        sites=sites,
        models=models,
        predictors=predictors,
        split='test',
        distribution='laplace',
        overwrite_existing_models=True,
        overwrite_results=True)

Data preprocessing... - site: ATMOS_20230405_baseline
 - Loading site data


/home/otto/git/methane-gapfill-ml/fluxgapfill/preprocess/load.py:9: DtypeWarning: Columns (251,399) have mixed types. Specify dtype option on import or set low_memory=False.
  site_data = pd.read_csv(site_data_path, na_values=na_values)
  0%|                                                                                                                                                   | 0/100 [00:00<?, ?it/s]

 - Estimating artificial gap distribution...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [22:45<00:00, 13.66s/it]


 - Done estimating artificial gap distribution.
 - Done preprocessing data for site ATMOS_20230405_baseline.
 - Processed data written to /home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230405_baseline.



...

/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or re

/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/otto/mambaforge/envs/ch4-gap-ml/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or re

 - Done model training.

Model training...
 - site: ATMOS_20230405_baseline
 - model: ann
 - predictors: DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure
 - Training on 0/10...
 - Training on 1/10...
 - Training on 2/10...
 - Training on 3/10...
 - Training on 4/10...
 - Training on 5/10...
 - Training on 6/10...
 - Training on 7/10...
 - Training on 8/10...
 - Training on 9/10...
 - Done model training.

Model testing...
 - site: ATMOS_20230405_baseline
 - model: rf
 - predictors: predictors
 - Writing test metrics to: /home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230405_baseline/models/rf/predictors/test_results.csv

Model testing...
 - site: ATMOS_20230405_baseline
 - model: ann
 - predictors: predictors
 - Writing test metrics to: /home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230405_baseline/models/ann/predictors/test_results.csv

Gapfilling: ATMOS_20230405_baseline, rf, predictors, laplace


/home/otto/git/methane-gapfill-ml/fluxgapfill/gapfill/gapfill.py:139: DtypeWarning: Columns (185,251,399) have mixed types. Specify dtype option on import or set low_memory=False.
  gap_df = pd.read_csv(site_dir / 'gap.csv')


ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
/home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230123_rf